In [137]:
import yfinance as yf
import pandas as pd
from backtesting import Backtest , Strategy

In [138]:
symbol = 'TRX-USD'
data = yf.Ticker(symbol)

price_data = data.history(start= '2023-12-23' ,interval = '15m')
price_data

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Datetime,,,,,,,
2023-12-23 00:00:00+00:00,0.105133,0.105203,0.105131,0.105203,0,0.0,0.0
2023-12-23 00:15:00+00:00,0.105204,0.105204,0.105061,0.105130,147568,0.0,0.0
2023-12-23 00:30:00+00:00,0.105130,0.105320,0.105130,0.105320,1613856,0.0,0.0
2023-12-23 00:45:00+00:00,0.105314,0.105352,0.105251,0.105251,492736,0.0,0.0
2023-12-23 01:00:00+00:00,0.105247,0.105317,0.105226,0.105264,0,0.0,0.0
...,...,...,...,...,...,...,...
2024-01-23 13:00:00+00:00,0.107352,0.107356,0.107113,0.107113,117792,0.0,0.0
2024-01-23 13:15:00+00:00,0.107122,0.107259,0.107122,0.107242,573152,0.0,0.0
2024-01-23 13:30:00+00:00,0.107251,0.107251,0.107144,0.107149,1652544,0.0,0.0


In [139]:
def MovingAverage (price, period):
    return pd.Series(price).rolling(period).mean()


In [140]:
class SMACross(Strategy):
    n_short = 10
    n_long = 20
    def init(self):
        close = self.data.Close
        self.ma_short = self.I(MovingAverage, close, self.n_short)
        self.ma_long = self.I(MovingAverage, close, self.n_long)
    def next(self):
        if self.ma_short[-2] < self.ma_long[-2]:
            if self.ma_short[-1] > self.ma_long[-1]:
                self.buy()
        elif self.ma_short[-2] > self.ma_long[-2]:
            if self.ma_short[-1] < self.ma_long[-1]:
                if self.position:
                    self.position.close()


In [141]:
bt= Backtest(price_data, SMACross, cash=1000, commission=0, exclusive_orders=True)
output = bt.run()
bt.plot()


/home/kasra/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/home/kasra/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


GridPlot(id='p4234', ...)

In [142]:
output

Start                     2023-12-23 00:00...
End                       2024-01-23 14:00...
Duration                     31 days 14:00:00
Exposure Time [%]                   57.500824
Equity Final [$]                  1061.259194
Equity Peak [$]                   1114.192788
Return [%]                           6.125919
Buy & Hold Return [%]                1.064985
Return (Ann.) [%]                   58.819164
Volatility (Ann.) [%]               50.912381
Sharpe Ratio                         1.155302
Sortino Ratio                        3.750612
Calmar Ratio                        12.373906
Max. Drawdown [%]                   -4.753484
Avg. Drawdown [%]                   -0.609209
Max. Drawdown Duration       10 days 22:30:00
Avg. Drawdown Duration        0 days 18:30:00
# Trades                                   80
Win Rate [%]                             37.5
Best Trade [%]                       6.688698
Worst Trade [%]                     -1.800026
Avg. Trade [%]                    